# Question 1

In [ ]:
# Question 1

# Question 2

## Import the relevant Libraries

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as ss
from bs4 import BeautifulSoup
import ast
from urllib.request import Request, urlopen
from itertools import combinations 

## Get header from .name file and add to dataset

In [ ]:
dataset = pd.read_csv('house-votes-84.data',header=None)
soup_link2 = BeautifulSoup(open('house-votes-84.names'))
table_header = soup_link2.find('p')

def before(value, a):
    pos_a = value.find(a)
    if pos_a == -1: return ""
    return value[0:pos_a]

def after(value, a):
    pos_a = value.rfind(a)
    if pos_a == -1: return ""
    adjusted_pos_a = pos_a + len(a)
    if adjusted_pos_a >= len(value): return ""
    return value[adjusted_pos_a:]

header = after(table_header.text,"7. Attribute Information:\n")
header = before(header,"\n8. Missing Attribute Values: Denoted by")
list_header = []
for i in range(header.count('\n')):
    if(i<10):
        buffer = after(header," "+str(i+1)+". ")
    else:
        buffer = after(header,str(i+1)+". ")
    buffer = before(buffer,":")
    list_header.append(str(buffer))

dataset.columns = list_header
dataset.head()

In [ ]:
dataset.shape

##### There are 434 rows and 17 columns 

In [ ]:
dataset.describe(include = 'object')

#### From the table above, we can see the basic data structure of dataset: 
#### All columns is categorical data
#### First column "republican" have 2 unique value which is (republican and democrat)
#### 16 other columns have 3 unique value which are y, n and ? (while ? is the missing value)

## Checking for missing values

#### Missing values is fill with "?" in the dataset so we have to take care of it


In [ ]:
## replace "?" value to most frequent values in each columns
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values = "?" , strategy = 'most_frequent',verbose=0)
imputer = imputer.fit(dataset.iloc[:,1:])
dataset.iloc[:, 1:] = imputer.transform(dataset.iloc[:, 1:])
display(dataset.head())

#### So that all the missing value is fill with the most frequent values
#### Because there only 2 unique values for all columns so we don't need dummy variable

## Calculate Correlation Matrix of dataset to check correlations among columns 

#### Because all columns of the dataset have categorical values so that we’re can not simply use corr() function of Pandas dataframe. we're looking for other measure of association between two categorical features.
#### By using Cramér’s V correlation which based on a nominal variation of Pearson’s Chi-Square Test will help us to handle this scenario

In [ ]:
def cramers_v(confusion_matrix):
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2/n
    r,k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))    
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min( (kcorr-1), (rcorr-1)))

cols = list(dataset.columns)
corrM = np.zeros((len(cols),len(cols)))
for col1, col2 in combinations(cols, 2):
    idx1, idx2 = cols.index(col1), cols.index(col2)
    corrM[idx1, idx2] = cramers_v(pd.crosstab(dataset[col1], dataset[col2]))
    corrM[idx2, idx1] = corrM[idx1, idx2]

corr = pd.DataFrame(corrM, index=cols, columns=cols)

## Visualize Correlation Matrix of dataset by heat map

In [ ]:
sns.set(font_scale=2)
fig, ax = plt.subplots(figsize=(30, 30))
ax = sns.heatmap(corr, annot=True, ax=ax); ax.set_title("Cramer V Correlation between Variables");

## Selecting features based on correlation

#### We compare the correlation between features and remove one of two features that have a correlation higher than 0.9

In [ ]:
columns = np.full((corr.shape[0],), True, dtype=bool)
for i in range(corr.shape[0]):
    for j in range(i+1, corr.shape[0]):
        if corr.iloc[i,j] >= 0.9:
            if columns[j]:
                columns[j] = False
                selected_columns = dataset.columns[columns]
                new_dataset = dataset[selected_columns]
remove_columns = (list(set(dataset.columns) - set(new_dataset.columns)))
print(str(len(remove_columns))+" column removed which is: "+str(remove_columns))

#### Now dataset has only those columns with correlation less than 0.9

In [ ]:
display(new_dataset.head())

## Calculate distribution of selected features

#### We create new distribution dataframe for calculating

In [ ]:
buffer = pd.crosstab(index = new_dataset["Class Name"],columns=new_dataset["handicapped-infants"])
buffer = buffer.stack()
buffer.index = ['_'.join(idx) for idx in buffer.index]
buffer.name = "handicapped-infants"
distribution = pd.DataFrame(buffer)
for col in new_dataset.columns:
    if (col == "Class Name") or (col == "handicapped-infants"):
        pass
    else:
        buffer = pd.crosstab(index = new_dataset["Class Name"],columns=new_dataset[col])
        buffer = buffer.stack()
        buffer.index = ['_'.join(idx) for idx in buffer.index]
        distribution[col] = buffer.values 
display(distribution)

## Plotting the data to visualize their distribution

#### Using stacked bar to plot distribution

In [ ]:
sns.set(font_scale=1.5)
distribution.T.plot(kind='bar', stacked=True,figsize=(20,8),label='big')

# Question 3

#### Import libraries

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
# Import here

#### Encode the dataset

In [ ]:
lb = LabelBinarizer()
oe = LabelEncoder()
training_dataset = dataset.copy()
dataset_features = training_dataset.drop(columns='Class Name')
dataset_classname = training_dataset['Class Name']
X = dataset_features.copy()
for col in X:
    X[col] = oe.fit_transform(X[col])
Y = lb.fit_transform(dataset_classname)

## Decision Tree C4.5

### How to apply Decision Tree C4.5

Let's take 20 random records of our dataset to perform the Decision Tree C4.5 Classification

| Class Name | handicapped-infants | water-project-cost-sharing | adoption-of-the-budget-resolution | physician-fee-freeze | el-salvador-aid | religious-groups-in-schools | anti-satellite-test-ban | aid-to-nicaraguan-contras | mx-missile | immigration | synfuels-corporation-cutback | education-spending | superfund-right-to-sue | crime | duty-free-exports | export-administration-act-south-africa |
| ---------- | ------------------- | -------------------------- | --------------------------------- | -------------------- | --------------- | --------------------------- | ----------------------- | ------------------------- | ---------- | ----------- | ---------------------------- | ------------------ | ---------------------- | ----- | ----------------- | -------------------------------------- |
| democrat   | n                   | y                          | y                                 | n                    | y               | y                           | n                       | n                         | n          | n           | y                            | n                  | y                      | y     | n                 | n                                      |
| democrat   | n                   | y                          | y                                 | n                    | y               | y                           | n                       | n                         | n          | n           | y                            | n                  | y                      | n     | n                 | y                                      |
| democrat   | n                   | y                          | y                                 | n                    | y               | y                           | n                       | n                         | n          | n           | n                            | n                  | y                      | y     | y                 | y                                      |
| democrat   | n                   | y                          | n                                 | y                    | y               | y                           | n                       | n                         | n          | n           | n                            | n                  | y                      | y     | y                 | y                                      |
| democrat   | n                   | y                          | y                                 | n                    | n               | n                           | y                       | y                         | y          | n           | n                            | n                  | y                      | n     | n                 | y                                      |
| democrat   | y                   | y                          | y                                 | n                    | y               | y                           | n                       | n                         | n          | n           | y                            | n                  | y                      | y     | y                 | y                                      |
| democrat   | y                   | y                          | y                                 | n                    | n               | n                           | y                       | y                         | y          | n           | n                            | n                  | n                      | n     | n                 | y                                      |
| democrat   | y                   | y                          | y                                 | n                    | n               | y                           | y                       | y                         | y          | y           | y                            | n                  | n                      | n     | y                 | y                                      |
| democrat   | y                   | n                          | y                                 | n                    | n               | y                           | n                       | y                         | y          | y           | y                            | y                  | y                      | n     | n                 | y                                      |
| democrat   | y                   | y                          | y                                 | n                    | n               | n                           | y                       | y                         | y          | n           | n                            | n                  | y                      | n     | y                 | y                                      |
| democrat   | y                   | y                          | y                                 | n                    | n               | n                           | y                       | y                         | y          | n           | y                            | n                  | n                      | n     | y                 | y                                      |
| republican | n                   | y                          | n                                 | y                    | y               | y                           | n                       | n                         | n          | y           | n                            | y                  | y                      | y     | n                 | y                                      |
| republican | n                   | y                          | n                                 | y                    | y               | y                           | n                       | n                         | n          | n           | n                            | y                  | y                      | y     | n                 | y                                      |
| republican | n                   | y                          | n                                 | y                    | y               | y                           | n                       | n                         | n          | n           | n                            | n                  | y                      | y     | n                 | y                                      |
| republican | n                   | y                          | n                                 | y                    | y               | y                           | n                       | n                         | n          | n           | n                            | y                  | y                      | y     | n                 | y                                      |
| republican | n                   | y                          | n                                 | y                    | y               | n                           | n                       | n                         | n          | n           | n                            | n                  | y                      | y     | n                 | n                                      |
| republican | n                   | y                          | n                                 | y                    | y               | y                           | n                       | n                         | n          | n           | y                            | n                  | y                      | y     | n                 | y                                      |
| republican | n                   | y                          | n                                 | y                    | y               | y                           | n                       | n                         | n          | n           | n                            | y                  | y                      | y     | n                 | y                                      |
| republican | n                   | y                          | n                                 | y                    | y               | y                           | n                       | n                         | n          | y           | n                            | y                  | y                      | y     | n                 | y                                      |
| republican | n                   | y                          | n                                 | y                    | y               | y                           | n                       | n                         | n          | n           | n                            | n                  | y                      | y     | n                 | n                                      |

The first step, we need to calculate the Expected information (entropy) of "Class Name":

To do that we need to find out the probability of each class in "Class Name"

|| Repulican | Democrat|
| :---- |:----:| :----:|
| Occurrence| 9 | 11 |
| Probability | 0.45 | 0.55 |

Then we can calculate the **entropy** of "Class Name":

\begin{equation*}
Info(Class Name) = \sum_{i=1}^m p_i log_2 (p_i) = -0.45 * log_2 0.45 - 0.55 * log_2 0.55 = 0.99
\end{equation*}

To select the root node, we need to calculate the Gain Ratio of all attributes

Let's take the "handicapped-infants" for an example:

| Class Name | handicapped-infants |
| ---------- | ------------------- |
| democrat   | n                   |
| democrat   | n                   |
| democrat   | n                   |
| democrat   | n                   |
| democrat   | n                   |
| democrat   | y                   |
| democrat   | y                   |
| democrat   | y                   |
| democrat   | y                   |
| democrat   | y                   |
| democrat   | y                   |
| republican | n                   |
| republican | n                   |
| republican | n                   |
| republican | n                   |
| republican | n                   |
| republican | n                   |
| republican | n                   |
| republican | n                   |
| republican | n                   |

We need to find the occurences of each attribute values and its information needed (after using "handicapped-infants" to split "Class Name") for each class:

| handicapped-infants | republican (R) | democrat (D) | I(R, D) |
| :-----------------: | :------------: | :----------: | :------ |
| yes                 | 0              | 6            | \begin{equation*} -\frac{0}{6} log_2 \frac{0}{6} -\frac{6}{6} log_2 \frac{6}{6} = 0 \end{equation*}|
| no                  | 9              | 5            | \begin{equation*} -\frac{9}{14} log_2 \frac{9}{14} -\frac{9}{14} log_2 \frac{9}{14} = 0.94 \end{equation*}|

Information needed of handicapped-infants is
$$
Info (handicapped-infants) = \frac {6}{20} * 0 + \frac {14}{20} * 0.94 = 0.66 
$$

Then we calculate the Information gained of handicapped-infants:

$$
Gain (handicapped-infants) = Entropy(0.45, 0.64) - Info(handicapped-infants) = 0.99 - 0.66 = 0.33
$$

Split Information of handicapped-infants:

$$
Split (handicapped-infants) = -\frac{6}{20} log_2 \frac{6}{20} -\frac{14}{20} log_2 \frac{14}{20} = 0.88
$$

Finally, we can compute the Gain Ratio of handicapped-infants:

$$
Gain_Ratio (handicapped-infants) = \frac {Gain_info (handicapped-infants)} {Split (handicapped-infants)} = \frac {0.33}{0.88} = 0.38
$$


Do it continuosly for all others attributes, we have:

|                    | handicapped-infants | water-project-cost-sharing | adoption-of-the-budget-resolution | physician-fee-freeze | el-salvador-aid | religious-groups-in-schools | anti-satellite-test-ban | aid-to-nicaraguan-contras | mx-missile | immigration | synfuels-corporation-cutback | education-spending | superfund-right-to-sue | crime | duty-free-exports | export-administration-act-south-africa |
| ------------------ | ------------------- | -------------------------- | --------------------------------- | -------------------- | --------------- | --------------------------- | ----------------------- | ------------------------- | ---------- | ----------- | ---------------------------- | ------------------ | ---------------------- | ----- | ----------------- | -------------------------------------- |
| I(R, D)yes         | 0.00                | 1.00                       | 0.00                              | 0.47                 | 0.94            | 1.00                        | 0.00                    | 0.00                      | 0.00       | 1.00        | 0.59                         | 0.65               | 1.00                   | 0.89  | 0.00              | 0.98                                   |
| I(R, D)no          | 0.94                | 0.00                       | 0.47                              | 0.00                 | 0.00            | 0.72                        | 0.97                    | 0.94                      | 0.94       | 0.99        | 0.96                         | 0.86               | 0.00                   | 0.00  | 0.94              | 0.92                                   |
| Information Needed | 0.66                | 0.95                       | 0.23                              | 0.23                 | 0.66            | 0.93                        | 0.73                    | 0.66                      | 0.66       | 0.99        | 0.83                         | 0.80               | 0.85                   | 0.58  | 0.66              | 0.97                                   |
| Information gained | 0.33                | 0.04                       | 0.76                              | 0.76                 | 0.33            | 0.06                        | 0.26                    | 0.33                      | 0.33       | 0.00        | 0.16                         | 0.19               | 0.14                   | 0.41  | 0.33              | 0.02                                   |
| Split Information  | 0.88                | 0.29                       | 1.00                              | 1.00                 | 0.88            | 0.81                        | 0.81                    | 0.88                      | 0.88       | 0.72        | 0.93                         | 0.88               | 0.61                   | 0.93  | 0.88              | 0.61                                   |
| Gain Ratio         | 0.38                | 0.16                       | 0.76                              | 0.76                 | 0.38            | 0.08                        | 0.33                    | 0.38                      | 0.38       | 0.00        | 0.17                         | 0.22               | 0.24                   | 0.44  | 0.38              | 0.04                                   |

The attribute with the maximum gain ratio (adoption-of-the-budget-resolution) is selected as the splitting attribute.

The root node now declared and split into 2 branchs (yes, no), for each branch, we redo above step to find the next node and keep going.

### Decision Tree programming

In [ ]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from graphviz import Source

dtModel = DecisionTreeClassifier(criterion="entropy")
dtModel.fit(X, Y)

tree = export_graphviz(dtModel, out_file=None, label='root', feature_names=dataset_features.columns, class_names=lb.classes_, proportion=True, filled=True)
s = Source(tree, format="png")
s.view()

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(dtModel, X, Y, cv=10, scoring="f1").mean()

## Naive Bayes

#### Create classifier

In [ ]:
# Naive Bayes classifier can be found in sklearn.naive_bayes
from sklearn.naive_bayes import GaussianNB

nbModel = GaussianNB()
cross_val_score(nbModel, X, Y, cv=10).mean()

## ZeroR

In ZeroR, the algorithm finds most frequent class name and return that prediction constantly   
Because the dataset has 267 'democrat' records and 168 'republican' records  
The accuracy of this ZeroR model should be **Acc = (267)/(267+168) ≈ 0.6138**

In [ ]:
# ZeroR classifier (DummyClassifier) can be found in sklearn.dummy packages
from sklearn.dummy import DummyClassifier
zeroRModel = DummyClassifier(strategy='most_frequent')

zeroRModel.fit(X, dataset_classname)
print("ZeroR accuracy:", zeroRModel.score(X, dataset_classname))

# Question 4

#### Precision and Recall

In [ ]:
import numpy as np
from sklearn import svm
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import precision_score, recall_score

def calc_precision_recall_f1(model, x, y, K=10):
    skf = KFold(n_splits=K, shuffle=True)
    precision_scores = []
    recall_scores = []
    f1_scores = []
    for train_index, test_index in skf.split(x, y):
        X_train, X_test = x[train_index], x[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = 2 * (precision * recall) / (precision + recall)
        
        precision_scores.append(precision)
        recall_scores.append(recall)
        f1_scores.append(f1)
    
    return precision_scores, recall_scores, f1_scores

In [ ]:
_,_,f1 = calc_precision_recall_f1(dtModel, X, dataset_classname)    #Precicion and Recall for Naive Bayes Classfier
np.mean(f1)

# END